In [1]:
import os
import pandas as pd
import csv
import numpy as np
import natsort
import matplotlib.pyplot as plt
from scipy import interpolate
import math

In [2]:
def alignwind(wind_df):
    Ub = wind_df['u'].mean()
    Vb = wind_df['v'].mean()
    Wb = wind_df['w'].mean()
    Sb = math.sqrt((Ub**2)+(Vb**2))
    beta = math.atan2(Wb,Sb)
    alpha = math.atan2(Vb,Ub)
    x1 = wind_df.index
    x = np.array(x1)
    Ur = wind_df['u']*math.cos(alpha)*math.cos(beta)+wind_df['v']*math.sin(alpha)*math.cos(beta)+wind_df['w']*math.sin(beta)
    Ur_arr = np.array(Ur)
    Vr = wind_df['u']*(-1)*math.sin(alpha)+wind_df['v']*math.cos(alpha)
    Vr_arr = np.array(Vr)
    Wr = wind_df['u']*(-1)*math.cos(alpha)*math.sin(beta)+wind_df['v']*(-1)*math.sin(alpha)*math.sin(beta)+wind_df['w']*math.cos(beta)     
    Wr_arr = np.array(Wr)
    T_arr = np.array(wind_df['T'])
    u_arr = np.array(wind_df['u'])
    v_arr = np.array(wind_df['v'])
    w_arr = np.array(wind_df['w'])
    
    fu_p = interpolate.interp1d(x, Ur_arr,fill_value="extrapolate") #interpolate u_p
    fv_p = interpolate.interp1d(x, Vr_arr,fill_value="extrapolate") #interpolate v_p
    fw_p = interpolate.interp1d(x, Wr_arr,fill_value="extrapolate") #interpolate w_p
    ft = interpolate.interp1d(x, T_arr,fill_value="extrapolate") #interpolate T (aka T_p)
    fu = interpolate.interp1d(x, u_arr,fill_value="extrapolate") #interpolate u
    fv = interpolate.interp1d(x, v_arr,fill_value="extrapolate") #interpolate v
    fw = interpolate.interp1d(x, w_arr,fill_value="extrapolate") #interpolate w
    
    RMY_xnew = np.arange(0, 24000)
    RMY_ynew_up = fu_p(RMY_xnew)   # use interpolation function returned by `interp1d`
    RMY_ynew_vp = fv_p(RMY_xnew)
    RMY_ynew_wp = fw_p(RMY_xnew)
    RMY_ynew_t = ft(RMY_xnew)
    RMY_ynew_u = fu(RMY_xnew)
    RMY_ynew_v = fv(RMY_xnew)
    RMY_ynew_w = fw(RMY_xnew)
    
    S = Sb
    b = beta*180/math.pi
    a = alpha*180/math.pi
    df = pd.DataFrame({'base_index':RMY_xnew,'u':RMY_ynew_u, 'v':RMY_ynew_v, 'w':RMY_ynew_u,'u_p':RMY_ynew_up, 'v_p':RMY_ynew_vp, 'w_p':RMY_ynew_wp, 'T':RMY_ynew_t})
    return S,b,a,df

In [3]:
def runFunc(filePath):
    RMY_UpWp = []
    RMY_UpWpmean1 = []
    
    WM1_UpWp = []
    WM1_UpWpmean1 = []
    
    WM2_UpWp = []
    WM2_UpWpmean1 = []
    
    R3_UpWp = []
    R3_UpWpmean1 = []
    
    for root, dirnames, filenames in os.walk(filePath):
        for filename in natsort.natsorted(filenames):
            file = os.path.join(root, filename)
            if file.endswith(".S07"):
                RMY_df = pd.read_fwf(file, index_col=None, header = None)
                RMY_df.columns =['u', 'v', 'w', 'T', 'err_code']
                S,b,a,df = alignwind(RMY_df)
                RMY_up = df['u_p']
                RMY_wp = df['w_p']
                for i in range(len(RMY_up)):
                    RMY_UpWp_i = (RMY_up[i]-RMY_up.mean())*RMY_wp[i]
                    RMY_UpWp.append(RMY_UpWp_i)
                RMY_UpWp_arr = np.array(RMY_UpWp)
                RMY_mean_i = RMY_UpWp_arr.mean()
                RMY_UpWpmean1.append(RMY_mean_i)
                RMY_UpWp_arr.clear()
                
            elif file.endswith(".S08"):
                WM1_df = pd.read_csv(file, index_col=None, header = None)
                WM1_df.columns = ['node_ad', 'u', 'v', 'w', 'units',"T", 'err_code','chk_sum']
                S,b,a,df = alignwind(WM1_df)
                WM1_up = df['u_p']
                WM1_wp = df['w_p']
                for i in range(len(WM1_up)):
                    WM1_UpWp_i = (WM1_up[i]-WM1_up.mean())*WM1_wp[i]
                    WM1_UpWp.append(WM1_UpWp_i)
                WM1_UpWp_arr = np.array(WM1_UpWp)
                WM1_mean_i = np.nanmean(WM1_UpWp_arr)
                WM1_UpWpmean1.append(WM1_mean_i)
                WM1_UpWp_arr.clear()
            elif file.endswith(".S09"):
#for WindMaster
#                 WM2_df = pd.read_csv(file, index_col=None, header = None)
#                 WM2_df.columns = ['node_ad', 'u', 'v', 'w', 'units',"T", 'err_code','chk_sum']
#                 S,b,a,df = alignwind(WM2_df)
#                 WM2_up = df['u_p']
#                 WM2_wp = df['w_p']
#                 for i in range(len(WM2_up)):
#                     WM2_UpWp_i = (WM2_up[i]-Wm2_up.mean())*WM2_wp[i]
#                     WM2_UpWp.append(WM2_UpWp_i)
#                 WM2_UpWp_arr = np.array(WM2_UpWp)
#                 WM2_mean_i = np.nanmean(WM2_UpWp_arr)
#                 WM2_UpWpmean1.append(WM2_mean_i)
#for CSAT
                WM2_df = pd.read_csv(file, index_col=None, header = None)
                WM2_df.columns = ['u', 'v', 'w', "T", '?']
                S,b,a,df = alignwind(WM2_df)
                WM2_up = df['u_p']
                WM2_wp = df['w_p']
                for i in range(len(WM2_up)):
                    WM2_UpWp_i = (WM2_up[i]-WM2_up.mean())*WM2_wp[i]
                    WM2_UpWp.append(WM2_UpWp_i)
                WM2_UpWp_arr = np.array(WM2_UpWp)
                WM2_mean_i = np.nanmean(WM2_UpWp_arr)
                WM2_UpWpmean1.append(WM2_mean_i)
                WM2_UpWp_arr.clear()

            elif file.endswith(".S10"):
                R3_df = pd.read_csv(file, index_col=None, header = None)
                R3_df.columns = ['col1','col2', 'u', 'v', 'w', 'T','chk_sum']
                S,b,a,df = alignwind(R3_df)
                R3_up = df['u_p']
                R3_wp = df['w_p']
                for i in range(len(R3_up)):
                    R3_UpWp_i = (R3_up[i]-R3_up.mean())*R3_wp[i]
                    R3_UpWp.append(R3_UpWp_i)
                R3_UpWp_arr = np.array(R3_UpWp)
                R3_mean_i = R3_UpWp_arr.mean()
                R3_UpWpmean1.append(R3_mean_i)
                R3_UpWp_arr.clear()
    return RMY_UpWpmean1, WM1_UpWpmean1, WM2_UpWpmean1, R3_UpWpmean1

In [6]:
RMY_UpWpmean1, WM1_UpWpmean1, WM2_UpWpmean1, R3_UpWpmean1 = runFunc(r"F:\benchTest_data2\analysis_copy")
print("'tis done")

'tis done


In [7]:
os.chdir(r"F:\benchTest_data2")
R3fin_df = pd.DataFrame(data = R3_UpWpmean1,columns=['R3_UpWp_mean'])
R3fin_df.to_csv("R3_UpWp_mean_final_CSAT_TEST_ALL.csv")
RMYfin_df = pd.DataFrame(data = RMY_UpWpmean1,columns=['RMY_UpWp_mean'])
RMYfin_df.to_csv("RMY_UpWp_mean_final_CSAT_TEST_ALL.csv")
WM1fin_df = pd.DataFrame(data = WM1_UpWpmean1,columns=['WM1_UpWp_mean'])
WM1fin_df.to_csv("WM1_UpWp_mean_final_CSAT_TEST_ALL.csv")
WM2fin_df = pd.DataFrame(data = WM2_UpWpmean1,columns=['WM2_UpWp_mean'])
WM2fin_df.to_csv("WM2_UpWp_mean_final_CSAT_TEST_ALL.csv")